# **Economic Freedom and Happiness Across Europe**

In [1]:
import pandas as pd
import plotly.io as pio
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pycountry 

import subprocess
import sys


from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Function for converting to iso 
def getIsoCode(country_name):
    try:
        country_iso = pycountry.countries.search_fuzzy(country_name)[0]
        return country_iso.alpha_3
    except LookupError:
        return ""

In [2]:
%%html
<style>
.cartesianlayer .bars path,
.colorbar .cbfills,
.slider-rail-rect { clip-path: inset(0% 0% 0% 0% round 6px); }

.legendundefined { clip-path: inset(0% 0% 0% 0% round 3px); }

.gtitle .line:nth-of-type(2) { fill: grey; }

.output_area:last-of-type,
.cell_output {
    filter: drop-shadow(0px 0px 10px rgba(0, 0, 0, 0.2));
    -webkit-backdrop-filter: blur(0); // fixes a safari drop-shadow bug
}

#notebook-container img,
main img {
    box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.2);
}

#notebook-container img,
main img,
.plotly-graph-div {
    border-radius: 10px;
    overflow: hidden;
    border: 1px solid black;
}
</style>

In [3]:
happiness_data = pd.read_csv("happiness.csv")
happiness_data["Country"] = happiness_data["Country or region"]
happiness_data = happiness_data.drop(columns=["Country or region", "Overall rank"])
happiness_data = happiness_data.rename(columns={"Score": "Happiness Score"})

In [4]:
freedom_data = pd.read_csv("freedom.csv", low_memory=False)
# Normalize country column
freedom_data = freedom_data.rename(columns={"countries": "Country"})

# Filter out years other than 2019
freedom_data["year"] = pd.to_numeric(freedom_data["year"], errors='coerce')
freedom_data = freedom_data[freedom_data["year"] == 2019]

# Reset and Drop
freedom_data = freedom_data.reset_index()
freedom_data = freedom_data.drop(columns="index")

freedom_filtered = freedom_data[freedom_data['Country'].isin([
    "Albania", "Andorra", "Austria", "Belarus", "Belgium", 
    "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", 
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", 
    "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", 
    "Liechtenstein", "Lithuania", "Luxembourg", "Malta", "Moldova", "Monaco", 
    "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal", 
    "Romania", "Russia", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", 
    "Sweden", "Switzerland", "Ukraine", "United Kingdom", "Vatican City"
])]

columns_to_keep = [
    'Country', 'ef_government', 'ef_legal', 'ef_money', 'ef_trade', 'ef_regulation', 'ef_rank'
]

freedom_data = freedom_filtered[columns_to_keep]

freedom_data = freedom_data.rename(columns={
    "ef_government": "Size of Government",
    "ef_legal": "Legal System and Property Rights",
    "ef_money": "Access to Sound Money",
    "ef_trade": "Freedom to Trade Internationally",
    "ef_regulation": "Regulation of Credit, Labor, and Business",
    "ef_rank": "Economic Freedom Rank"
})

print(freedom_data.columns)


Index(['Country', 'Size of Government', 'Legal System and Property Rights',
       'Access to Sound Money', 'Freedom to Trade Internationally',
       'Regulation of Credit, Labor, and Business', 'Economic Freedom Rank'],
      dtype='object')


In [5]:
data = pd.merge(happiness_data, freedom_data, on='Country')
data = data.drop_duplicates(subset=['Country'])

data["iso"] = data['Country'].apply(getIsoCode)

# CREATING NEW RANK
sorted_df = data.sort_values(by='Economic Freedom Rank')
# Display the sorted DataFrame with ranks
data = sorted_df

## Analysis Across Countries

In [6]:
import plotly.express as px

# Merge data with latitude and longitude
custom_colors = [
    [0.0, "#ffffff"],
    [1.0, "#c85fac"],
]

# Create the choropleth map focusing on Europe
fig = px.choropleth(data, locations="iso",
                    title='<b>Economic Freedom In Europe</b>',
                    locationmode='ISO-3',  # Use ISO-3 country codes
                    color="Happiness Score",
                    hover_name="iso",
                    width=800,
                    height=500,
                    hover_data=['iso', 'Happiness Score'],
                    color_continuous_scale=custom_colors,  # Adjust color scale as needed
                    scope='europe',  # Focus on Europe
                    center={"lat": 51.1657, "lon": 10.4515},  # Center of Europe (approximate)
                    projection='natural earth',
                    labels={'Happiness Score': 'Overall Happiness Score', 'iso': 'Country Code'},
                    )

fig.update_layout(title_x=0.5, margin=dict(b=0, l=0))

# Show the plot
fig.show()


## Top vs. Bottom performers

In [7]:
top_5 = data.nlargest(5, 'Happiness Score')
bottom_5 = data.nsmallest(5, 'Happiness Score')

# Create trace for top 5 countries
trace_top = go.Bar(
    x=top_5['Country'],
    y=top_5['Happiness Score'],
    name='Top 5 Happiest Countries',
    marker=dict(color='#049b9e')
)

# Create trace for bottom 5 countries
trace_bottom = go.Bar(
    x=bottom_5['Country'],
    y=bottom_5['Happiness Score'],
    name='Bottom 5 Happiest Countries',
    marker=dict(color='#c85fac')
)

# Define layout
layout = go.Layout(
    title='<b>Highest and Lowest Happiness Scores</b>',
    xaxis=dict(title='Country'),
    yaxis=dict(title='Happiness Score', range=[0, 10]),  # Set the same y-axis range for both plots
    barmode='group',  # Display bars side by side
    bargap=0.2,  # Gap between bars
    bargroupgap=0.1,  # Gap between groups of bars
    width=800,
    height=500,
)

#c85fac, #049b9e
# Create figure object
fig = go.Figure(data=[trace_top, trace_bottom], layout=layout)
fig.update_layout(title_x=0.5)
# Show the plot

fig.show()

## Dimensional Analysis

In [8]:
columns_to_keep=['GDP per capita', 'Social support',
       'Healthy life expectancy', 'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Size of Government',
       'Legal System and Property Rights', 'Access to Sound Money', 'Freedom to Trade Internationally', 'Regulation of Credit, Labor, and Business']

corr = data[columns_to_keep].corr().round(1)
color_continuous_scale='tropic'

fig = px.imshow(corr,
                x=corr.index,
                y=corr.columns,
                color_continuous_scale='tropic',
                zmin=-1, zmax=1,
                labels=dict(color='Correlation'),
                text_auto=True)

fig.update_layout(title='<b>Correlation Matrix of Economic Freedom and Happiness Indicators</b>',
                  xaxis_title='',
                  yaxis_title='',
                  title_x=0.5,
                  coloraxis_colorbar=dict(title='Correlation'),
                  font=dict(family='Arial', size=12),
                  width=800, height=600),

fig.update_traces(textfont_size=10, hoverinfo='text')
fig.show()



In [9]:
# Convert data to numpy arrays
X = np.array(data['Legal System and Property Rights'])  # Independent variable
y = np.array(data['Happiness Score'])  # Dependent variable

# Compute coefficients of linear regression (y = mx + c)
X_mean = np.mean(X)
y_mean = np.mean(y)
m = np.sum((X - X_mean) * (y - y_mean)) / np.sum((X - X_mean) ** 2)
c = y_mean - m * X_mean

# Predicted values of y based on the regression equation
y_pred = m * X + c

# Plotting the scatter plot and regression line using Plotly
fig = go.Figure()

# Scatter plot of data points
fig.add_trace(go.Scatter(
    x=X,
    y=y,
    mode='markers',
    marker=dict(color='#c85fac'),
    name='Data Points'
))

# Regression line
fig.add_trace(go.Scatter(
    x=X,
    y=y_pred,
    mode='lines',
    line=dict(color='black', width=2),
    name='Regression Line'
))

# Layout settings with adjusted axis ranges
fig.update_layout(
    title='<b>Legal System and Property Rights vs. Happiness Score</b>',
    xaxis_title='Legal System and Property Rights',
    yaxis_title='Happiness Score',
    showlegend=False,
    hovermode='closest',
    template='plotly_white',
    width=800,
    height=500,

    yaxis_range=[0, 10],  # Set range for y-axis
)

# Show the plot
fig.update_layout(title_x=0.5)
fig.show()

## Ex-soviet States

In [10]:
import plotly.graph_objects as go

ex_soviet_states = ["Belarus", "Estonia", "Latvia", "Lithuania", "Moldova", "Russia", "Ukraine", "Bosnia and Herzegovina", "Albania"]

data['Ex-Soviet State'] = data['Country'].isin(ex_soviet_states)
data['Ex-Soviet State'] = data['Ex-Soviet State'].apply(lambda x: 'True' if x else 'False')

ex = data[data['Ex-Soviet State'] == "True"]
non_ex = data[data['Ex-Soviet State'] == "False"]

fig = go.Figure()

fig.add_trace(go.Box(
    y=ex["Happiness Score"],
    name="Former Soviet Union",
    fillcolor="#049b9e", 
    line_color="black",
))

fig.add_trace(go.Box(
    y=non_ex["Happiness Score"],
    name="Not Former Soviet Union",
    fillcolor="#c85fac",  
    line_color="black",  
))

fig.update_layout(
    yaxis_title="Happiness Score",
    width=800,
    height=500,
)

fig.show()

## Summary

## References

## External Links